$\newcommand{norm}[1]{||#1||}$
$\newcommand{bx}{\mathbf x}$
$\newcommand{bu}{\mathbf u}$
$\newcommand{d}{\text{div }}$
$\newcommand{f}{\mathbf f}$

# 线弹性问题

## 模型

$$
-\mu\Delta\bu - (\lambda+\mu)\nabla\d\bu = \f
$$

令
$$
\sigma(\bu) = \lambda\d\bu I+\mu(\nabla\bu + \nabla\bu^T)
$$

$$
-\d\sigma(\bu) = \f
$$

In [1]:
from IPython.display import display
from sympy import *
import numpy as np
init_printing()

def gradient(scalar_function, variables):
    matrix_scalar_function = Matrix([scalar_function])
    return matrix_scalar_function.jacobian(variables)

In [26]:
x, y, z, a, b, c, d, e = symbols('x, y, z, a, b, c, lam, mu')
u = x*(1-x)*y*(1-y)*z*(1-z)
mat = Matrix([a*u, b*u, c*u])
du = mat.jacobian((x, y, z))
print(du)
sigma = d*du.trace()*Matrix([[1, 0, 0],[0, 1, 0],[0, 0, 1]])+ e*(du + du.transpose())
f = -(diff(sigma, x)[:, 0] + diff(sigma, y)[:, 1] + diff(sigma, z)[:, 2])
t0, t1, t2, t3 = symbols('t0, t1, t2, t3')
sub = {(-x+1)*(-y+1)*(-z+1):t0, (-y+1)*(-z+1):t1, (-x+1)*(-z+1):t2, (-x+1)*(-y+1):t3}
ff = f.subs(sub)
print(ff)

Matrix([[-a*x*y*z*(-y + 1)*(-z + 1) + a*y*z*(-x + 1)*(-y + 1)*(-z + 1), -a*x*y*z*(-x + 1)*(-z + 1) + a*x*z*(-x + 1)*(-y + 1)*(-z + 1), -a*x*y*z*(-x + 1)*(-y + 1) + a*x*y*(-x + 1)*(-y + 1)*(-z + 1)], [-b*x*y*z*(-y + 1)*(-z + 1) + b*y*z*(-x + 1)*(-y + 1)*(-z + 1), -b*x*y*z*(-x + 1)*(-z + 1) + b*x*z*(-x + 1)*(-y + 1)*(-z + 1), -b*x*y*z*(-x + 1)*(-y + 1) + b*x*y*(-x + 1)*(-y + 1)*(-z + 1)], [-c*x*y*z*(-y + 1)*(-z + 1) + c*y*z*(-x + 1)*(-y + 1)*(-z + 1), -c*x*y*z*(-x + 1)*(-z + 1) + c*x*z*(-x + 1)*(-y + 1)*(-z + 1), -c*x*y*z*(-x + 1)*(-y + 1) + c*x*y*(-x + 1)*(-y + 1)*(-z + 1)]])
Matrix([[4*a*mu*t1*y*z - lam*(-2*a*t1*y*z + b*t0*z - b*t1*x*z - b*t2*y*z + b*x*y*z*(-z + 1) + c*t0*y - c*t1*x*y - c*t3*y*z + c*x*y*z*(-y + 1)) - mu*(-2*a*t2*x*z + b*t0*z - b*t1*x*z - b*t2*y*z + b*x*y*z*(-z + 1)) - mu*(-2*a*t3*x*y + c*t0*y - c*t1*x*y - c*t3*y*z + c*x*y*z*(-y + 1))], [4*b*mu*t2*x*z - lam*(a*t0*z - a*t1*x*z - a*t2*y*z + a*x*y*z*(-z + 1) - 2*b*t2*x*z + c*t0*x - c*t2*x*y - c*t3*x*z + c*x*y*z*(-x + 1)) -

In [30]:
collect(expand(ff[0,0]), (t0, t1, t2, t3))